Download template MNI-HCP which Tian atlas is defined in from templateflow

In [ ]:
from templateflow import api
api.get('MNI152NLin6Asym', resolution=1, suffix='T1w')

Plot tian atlas in different scales

In [ ]:
%matplotlib qt
import matplotlib.pyplot as plt
from subcortex_visualization.plotting import plot_subcortical_data

for s in ["S1", "S2"]:
    plot_subcortical_data(atlas=f"Melbourne_{s}", hemisphere='both', cmap="hsv")
    fig = plt.gcf()
    ax = plt.gca()
    fig.patch.set_facecolor('black')
    ax.set_facecolor('black')         
    plt.show()

Visualizing connectome matrix

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt
from cmap import Colormap
from matplotlib.colors import LinearSegmentedColormap
from nilearn.plotting import plot_matrix
import pandas as pd
import numpy as np

In [27]:
df = pd.read_csv("/Users/payamsadeghishabestari/temp_folder/dti/asjt/conn/tian_S3_conn.csv", header=None)
cm = Colormap('colorbrewer:OrRd_9')
combined_cmap = LinearSegmentedColormap.from_list("", cm(np.linspace(0, 1, 500)), N=1000)

labels_fname = "./data/Tian2020MSA/3T/Subcortex-Only/Tian_Subcortex_S3_3T_label.txt"
labels = np.loadtxt(labels_fname, dtype=str)

fig, ax = plt.subplots(figsize=(10, 8))
plot_matrix(df, labels=labels, cmap=combined_cmap, axes=ax, colorbar=True, vmax=0.9, vmin=0)

for label in ax.get_xticklabels()[:25]:
    label.set_color('lightgreen')
for label in ax.get_yticklabels()[:25]:
    label.set_color('lightgreen')

for label in ax.get_xticklabels()[25:]:
    label.set_color('lavender')
for label in ax.get_yticklabels()[25:]:
    label.set_color('lavender')

fig.patch.set_facecolor('black')
ax.set_facecolor('black')          

# Make grid lines white for visibility (optional)
for spine in ax.spines.values():
    spine.set_color('white')


plt.show()

Convert dataset into BIDS (anat and func)

In [5]:
import os
import shutil

# Paths
fMRI_path = "/Volumes/Extreme_SSD/payam_data/antinomics_data/fMRI"
sMRI_path = "/Volumes/Extreme_SSD/payam_data/antinomics_data/sMRI_T1"
bids_path = "/Volumes/Extreme_SSD/payam_data/antinomics_data/BIDS"

# Create BIDS dataset folder if it doesn't exist
os.makedirs(bids_path, exist_ok=True)

# Create minimal dataset_description.json
import json
dataset_description = {
    "Name": "My fMRI/sMRI Dataset",
    "BIDSVersion": "1.8.0",
    "DatasetType": "raw"
}
with open(os.path.join(bids_path, "dataset_description.json"), "w") as f:
    json.dump(dataset_description, f, indent=4)

# Helper to zero-pad subject IDs
def sub_id(idx):
    return f"sub-{idx:02d}"

# List of subjects based on sMRI filenames
subjects = [f.replace(".nii.gz", "") for f in sorted(os.listdir(sMRI_path)) if f.endswith(".nii.gz")]

for i, subj_name in enumerate(subjects, start=1):
    sub_folder = os.path.join(bids_path, sub_id(i))
    
    # Create anat folder
    anat_folder = os.path.join(sub_folder, "anat")
    os.makedirs(anat_folder, exist_ok=True)
    
    # Copy T1 (anat)
    t1_src = os.path.join(sMRI_path, f"{subj_name}.nii.gz")
    t1_dst = os.path.join(anat_folder, f"{sub_id(i)}_T1w.nii.gz")
    shutil.copy(t1_src, t1_dst)
    
    # Create func folder
    func_folder = os.path.join(sub_folder, "func")
    os.makedirs(func_folder, exist_ok=True)
    
    # Copy fMRI scans from all session folders into runs
    run_counter = 1
    for session_folder in ["s1", "s2"]:
        func_src_folder = os.path.join(fMRI_path, session_folder)
        func_files = [f for f in os.listdir(func_src_folder) if f.startswith(subj_name) and f.endswith(".nii.gz")]
        for func_file in sorted(func_files):
            func_src = os.path.join(func_src_folder, func_file)
            func_dst = os.path.join(func_folder, f"{sub_id(i)}_task-rest_run-{run_counter:02d}_bold.nii.gz")
            shutil.copy(func_src, func_dst)
            run_counter += 1